# Model creation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam



### Importing saved Dataset as numpy array

In [ ]:
with open("/content/drive/MyDrive/food_recommender/train_images.npy", "rb") as f:
  train_images = np.load(f)

with open("/content/drive/MyDrive/food_recommender/train_labels.npy", "rb") as f:
  train_labels = np.load(f)

train_images = preprocess_input(train_images)

### One-Hot Encoding the labels 

In [ ]:
# One-Hot Encode
shape = (train_labels.size, train_labels.max() + 1)
enc_train_labels = np.zeros(shape)
rows = np.arange(train_labels.size)

enc_train_labels[rows, train_labels] = 1

### Importing MobileNet Model

In [ ]:
mobilenet_model = MobileNet()

17235968/17225924 [==============================] - 0s 0us/step


In [ ]:
mobilenet_model.summary()

### Removing last 6 layers from the model and adding our own output layer
### Setting some layers in model to non trainable

In [ ]:
x = mobilenet_model.layers[-6].output
output_layer = Dense(units=80, activation='softmax')(x)
model = Model(inputs=mobilenet_model.input, outputs=output_layer)

for layer in model.layers[:-20]:
  layer.trainable = False

In [ ]:
model.summary()

### Compiling the new Model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training new Model

In [ ]:
model.fit(x=train_images, y=enc_train_labels, batch_size=32, epochs=16, validation_split=0.15)

### Saving the model in hd5f format

In [ ]:
model.save("/content/drive/MyDrive/food_recommender/63_food_recom.h5")